In [32]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize, LabelBinarizer
import sys
sys.path.append('../..')
from modules.many_features import utils, constants

In [33]:
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

#### The Datasets

In [34]:
train_df = pd.read_csv('../../final/data/train_set_basic.csv')
train_df = train_df.fillna(-1)
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]
X_train.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat
0,9.007012,-1.000000,-1.000000,3.519565,440.499323,103.442762,59.017997,2.612173,1,0.650757,114.794964,112.308159,25.612786,5.969710,116.026042,27.021037,13.397977
1,8.760976,-1.000000,0.491469,-1.000000,259.895852,103.885481,-1.000000,2.529991,0,0.728641,74.824352,-1.000000,-1.000000,-1.000000,-1.000000,26.282929,-1.000000
2,7.490324,70.812609,-1.000000,1.495604,482.109919,79.543391,-1.000000,2.824995,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,22.470972,-1.000000
3,8.113370,499.313483,3.507612,0.000000,195.351883,100.545858,203.895756,2.420797,1,0.860863,9.120947,41.131511,1.843890,18.845473,106.199806,24.340111,104.373581
4,13.935301,349.569415,5.190725,6.894195,489.595939,102.234294,150.085853,4.089225,1,0.216907,20.344863,92.547095,19.815123,29.543875,98.388710,41.805903,30.655044


In [35]:
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 17), (14000, 17), (56000,), (14000,))

In [36]:
y_test

array([5, 1, 4, ..., 2, 4, 6])

#### Feed-forward Neural Network - Keras

In [47]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from keras import backend as K
import keras

2022-11-24 19:29:00.269964: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-24 19:29:01.322522: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-24 19:29:01.322682: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-11-24 19:29:01.322692: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [48]:
#keras.utils.set_random_seed(42)

In [49]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [51]:
def numerize_labels(pd_series):
    series_copy = pd_series.copy()
    series_copy = series_copy.map(class_dict)
    return series_copy

y_train_num = y_train
y_test_num = y_test

dummy_y_train = np_utils.to_categorical(y_train_num)
dummy_y_test = np_utils.to_categorical(y_test_num)
dummy_y_train.shape, dummy_y_test.shape

((56000, 8), (14000, 8))

In [55]:
model = Sequential()
model.add(Dropout(0.5, input_shape=(17,)))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', f1_m, precision_m, recall_m])
history = model.fit(X_train_norm, dummy_y_train, epochs=100, batch_size=1024, shuffle=True, validation_split=0.20, verbose=0)
#model.save('../models/baselines/synthentic_nn.h5')
loss, accuracy, f1, precision, recall = model.evaluate(X_test_norm, dummy_y_test, verbose=0)
loss, accuracy, f1, precision, recall

(1.9412058591842651, 0.24364285171031952, 0.0, 0.0, 0.0)

In [56]:
#Alternatively
y_pred1_nn = model.predict(X_test_norm)
y_pred_nn = np.argmax(y_pred1_nn, axis=1)
np.unique(y_pred_nn)

438/438 [==============================] - 1s 2ms/step


array([0, 1, 2, 3, 4, 5, 6])

In [57]:
test_df_nn = pd.DataFrame()
test_df_nn['y_actual'] = y_test
test_df_nn['y_pred'] = y_pred_nn
test_df_nn.isna().sum()

y_actual    0
y_pred      0
dtype: int64

In [58]:
success_rate_nn, success_df_nn = utils.success_rate(test_df_nn)
success_rate_nn

24.364285714285714

In [59]:
acc_nn, f1_nn, roc_auc_nn = utils.test(test_df_nn['y_actual'], test_df_nn['y_pred'])
acc_nn, f1_nn, roc_auc_nn

(0.24364285714285713, 0.13938858912177068, 0.5682098894938374)

#### Feed-forward Neural Network - PyTorch

In [73]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from tqdm.notebook import tqdm

In [74]:
def get_class_distribution(obj):
    count_dict = dict(zip(list(range(8)), [0]*8))
    
    for i in obj:
        try:
            count_dict[i] += 1
        except:
            print('Check classes')
            
    return count_dict

In [75]:
class ClassifierDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [76]:
X_train_new, X_val, y_train_new, y_val = train_test_split(X_train, y_train, test_size=0.1, stratify=y_train, random_state=constants.SEED)

In [77]:
scaler = MinMaxScaler()
X_train_new = scaler.fit_transform(X_train_new)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
X_train_new, y_train_new = np.array(X_train_new), np.array(y_train_new)
X_val, y_val = np.array(X_val), np.array(y_val)
X_test, y_test = np.array(X_test), np.array(y_test)

In [78]:
train_dataset = ClassifierDataset(torch.from_numpy(X_train_new).float(), torch.from_numpy(np.array(y_train_new)).long())

val_dataset = ClassifierDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).long())

test_dataset = ClassifierDataset(torch.from_numpy(X_test).float(), torch.from_numpy(np.array(y_test)).long())

In [79]:
target_list = []
for _, t in train_dataset:
    target_list.append(t)
    
target_list = torch.tensor(target_list)

In [80]:
class_count = [i for i in get_class_distribution(y_train_new).values()]
class_weights = 1./torch.tensor(class_count, dtype=torch.float) 
print(class_weights)
###################### OUTPUT ######################tensor([0.1429, 0.0263, 0.0020, 0.0022, 0.0070, 0.0714])

tensor([0.0001, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002, 0.0002])


In [81]:
class_weights_all = class_weights[target_list]

In [82]:
weighted_sampler = WeightedRandomSampler(
    weights=class_weights_all,
    num_samples=len(class_weights_all),
    replacement=True
)

In [85]:
#EPOCHS = 300
EPOCHS = 11
BATCH_SIZE = 16
LEARNING_RATE = 0.0007
NUM_FEATURES = 17
NUM_CLASSES = 8

In [86]:
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, sampler=weighted_sampler)

val_loader = DataLoader(dataset=val_dataset, batch_size=1)

test_loader = DataLoader(dataset=test_dataset, batch_size=1)

In [87]:
class MulticlassClassification(nn.Module):
    def __init__(self, num_feature, num_class):
        super(MulticlassClassification, self).__init__()
        
        self.layer_1 = nn.Linear(num_feature, 512)
        self.layer_2 = nn.Linear(512, 128)
        self.layer_3 = nn.Linear(128, 64)
        self.layer_out = nn.Linear(64, num_class) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)
        self.batchnorm1 = nn.BatchNorm1d(512)
        self.batchnorm2 = nn.BatchNorm1d(128)
        self.batchnorm3 = nn.BatchNorm1d(64)
        
    def forward(self, x):
        x = self.layer_1(x)
        x = self.batchnorm1(x)
        x = self.relu(x)
        
        x = self.layer_2(x)
        x = self.batchnorm2(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_3(x)
        x = self.batchnorm3(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_out(x)
        
        return x

In [89]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [90]:
model = MulticlassClassification(num_feature = NUM_FEATURES, num_class=NUM_CLASSES)
model.to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
model

MulticlassClassification(
  (layer_1): Linear(in_features=17, out_features=512, bias=True)
  (layer_2): Linear(in_features=512, out_features=128, bias=True)
  (layer_3): Linear(in_features=128, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=8, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (batchnorm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [91]:
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    
    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    
    acc = torch.round(acc * 100)
    
    return acc

In [92]:
accuracy_stats = {
    'train': [],
    "val": []
}

loss_stats = {
    'train': [],
    "val": []
}

In [93]:
print("Begin training.")
for e in tqdm(range(1, EPOCHS+1)):
    
    # TRAINING
    train_epoch_loss = 0
    train_epoch_acc = 0
    model.train()
    for X_train_batch, y_train_batch in train_loader:
        X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
        optimizer.zero_grad()
        
        y_train_pred = model(X_train_batch)
        
        train_loss = criterion(y_train_pred, y_train_batch)
        train_acc = multi_acc(y_train_pred, y_train_batch)
        
        train_loss.backward()
        optimizer.step()
        
        train_epoch_loss += train_loss.item()
        train_epoch_acc += train_acc.item()
        
        
    # VALIDATION    
    with torch.no_grad():
        
        val_epoch_loss = 0
        val_epoch_acc = 0
        
        model.eval()
        for X_val_batch, y_val_batch in val_loader:
            X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            
            y_val_pred = model(X_val_batch)
                        
            val_loss = criterion(y_val_pred, y_val_batch)
            val_acc = multi_acc(y_val_pred, y_val_batch)
            
            val_epoch_loss += val_loss.item()
            val_epoch_acc += val_acc.item()
            
    loss_stats['train'].append(train_epoch_loss/len(train_loader))
    loss_stats['val'].append(val_epoch_loss/len(val_loader))
    accuracy_stats['train'].append(train_epoch_acc/len(train_loader))
    accuracy_stats['val'].append(val_epoch_acc/len(val_loader))
                              
    
    print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(val_loader):.5f} | Train Acc: {train_epoch_acc/len(train_loader):.3f}| Val Acc: {val_epoch_acc/len(val_loader):.3f}')

Begin training.


  0%|          | 0/11 [00:00<?, ?it/s]

Epoch 001: | Train Loss: 0.88767 | Val Loss: 0.41780 | Train Acc: 68.456| Val Acc: 86.411
Epoch 002: | Train Loss: 0.62505 | Val Loss: 0.31441 | Train Acc: 77.119| Val Acc: 90.232
Epoch 003: | Train Loss: 0.54297 | Val Loss: 0.27092 | Train Acc: 80.363| Val Acc: 91.625
Epoch 004: | Train Loss: 0.48832 | Val Loss: 0.23810 | Train Acc: 82.626| Val Acc: 92.804
Epoch 005: | Train Loss: 0.45219 | Val Loss: 0.22812 | Train Acc: 83.907| Val Acc: 92.911
Epoch 006: | Train Loss: 0.42645 | Val Loss: 0.23336 | Train Acc: 85.104| Val Acc: 92.839
Epoch 007: | Train Loss: 0.39978 | Val Loss: 0.21496 | Train Acc: 86.217| Val Acc: 92.982
Epoch 008: | Train Loss: 0.38251 | Val Loss: 0.21561 | Train Acc: 86.732| Val Acc: 93.018
Epoch 009: | Train Loss: 0.36437 | Val Loss: 0.21191 | Train Acc: 87.527| Val Acc: 93.036
Epoch 010: | Train Loss: 0.35004 | Val Loss: 0.19213 | Train Acc: 88.114| Val Acc: 93.982
Epoch 011: | Train Loss: 0.34282 | Val Loss: 0.21399 | Train Acc: 88.314| Val Acc: 93.464


In [94]:
y_pred_list = []
with torch.no_grad():
    model.eval()
    for X_batch, _ in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        _, y_pred_tags = torch.max(y_test_pred, dim = 1)
        y_pred_list.append(y_pred_tags.cpu().numpy())
y_pred_list = [a.squeeze().tolist() for a in y_pred_list]
y_pred_list[:5]

[7, 7, 7, 7, 7]

In [97]:
len(y_test), len(y_pred_list)

(14000, 14000)

In [98]:
test_df_tch = pd.DataFrame()
test_df_tch['y_actual'] = y_test
test_df_tch['y_pred'] = y_pred_list
test_df_tch.isna().sum()

y_actual    0
y_pred      0
dtype: int64

In [99]:
success_rate_tch, success_df_tch = utils.success_rate(test_df_tch)
success_rate_tch

9.6

In [ ]:
#11 epochs
acc = accuracy_score(y_test, y_pred_list)
f1 = f1_score(y_test, y_pred_list, average='macro', labels=np.unique(y_test))
roc_auc = multiclass_roc_auc_score(y_test, y_pred_list)
acc, f1, roc_auc

In [ ]:
#300 epochs
acc = accuracy_score(y_test, y_pred_list)
f1 = f1_score(y_test, y_pred_list, average='macro', labels=np.unique(y_test))
roc_auc = multiclass_roc_auc_score(y_test, y_pred_list)
acc, f1, roc_auc

In [ ]:
np.unique(y_test)

In [ ]:
y_test